In [1]:
import tensorflow as tf
import re, json, pickle
from tqdm import tqdm
import numpy as np
import tensorflow_hub as hub
from keras.utils.vis_utils import plot_model
from keras.layers import merge
import pathlib
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pre_path = "/content/drive/MyDrive/Special_Topic/Dataset/MSCOCO_2017/"
# image_path = "/content/drive/MyDrive/University/Special_Topic/Dataste/MSCOCO/MSCOCO_2017/val2017"
image_path =  pre_path + "val2017/"
anno_path = pre_path + '2017_annotations/captions_train2017.json'
glove_path = pre_path + 'GloVe.6B/glove.6B.300d.txt'


In [4]:
results_path = '/content/drive/MyDrive/Special_Topic/Show_Control_Tell/Finial/Demo4/Results/Show_Control_Tell_results_mode_2_0.plk'
with open(results_path, 'rb') as f:
  results = pickle.loads(f.read())
# pickle.dump(results, outfile)

In [5]:
len(results)

3945

In [6]:
results

[{'caption': 'A cat is laying on a bed with a cat.',
  'g_t': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
  'image_id': 504517},
 {'caption': 'A fire hydrant is sitting on a rock.',
  'g_t': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
  'image_id': 507068},
 {'caption': 'A table with a table and a table.',
  'g_t': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
  'image_id': 501377},
 {'caption': 'A baby elephant standing next to a baby elephant.',
  'g_t': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
  'image_id': 500214},
 {'caption': 'A plate of food with a plate of food.',
  'g_t': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
  'image_id': 507939},
 {'caption': 'A group of

# Remove g_t

In [7]:
for r in results:
  del r['g_t']

In [8]:
results[0]

{'caption': 'A cat is laying on a bed with a cat.', 'image_id': 504517}

# Evaluate

In [9]:
image_list = list()
error = list()
for i  in range(len(results)):
  if results[i]['image_id'] in image_list:
    error.append(i)
    continue
  image_list.append(results[i]['image_id'])
error.reverse()
for i in error:
  del results[i]

In [10]:
generated_caption_path = "generated_caption_val2017.json"
with open(generated_caption_path, 'w') as jsonfile:
  json.dump(results, jsonfile)
  # json.dump(results[225:], jsonfile)

In [11]:
!git clone https://github.com/davidfsemedo/coco-caption


Cloning into 'coco-caption'...
remote: Enumerating objects: 737, done.
remote: Total 737 (delta 0), reused 0 (delta 0), pack-reused 737
Receiving objects: 100% (737/737), 106.72 MiB | 30.33 MiB/s, done.
Resolving deltas: 100% (400/400), done.


In [12]:
import os, sys
sys.path.insert(0,"/content/coco-caption")

In [13]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [14]:
#  workaround provided by https://github.com/tylin/coco-caption/issues/26#issuecomment-439825144
# test_anno_path = pre_path + '2017_annotations/captions_val2017.json'
test_anno_path = pre_path + '2017_annotations/captions_train2017.json'
modified_anno_path = "/content/modified_captions_val2017.json"

with open(test_anno_path, 'r') as f:
    data = json.load(f)
    data['type'] = 'captions'
with open(modified_anno_path, 'w') as f:
    json.dump(data, f)

In [15]:
coco = COCO(modified_anno_path)
cocoRes = coco.loadRes(generated_caption_path)

loading annotations into memory...
0:00:01.235126
creating index...
index created!
Loading and preparing results...     
DONE (t=0.08s)
creating index...
index created!


In [16]:
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params['image_id'] = cocoRes.getImgIds()

# evaluate results
cocoEval.evaluate()

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 36239, 'reflen': 36441, 'guess': [36239, 32294, 28349, 24404], 'correct': [23307, 10439, 4134, 1431]}
ratio: 0.9944567931724982
Bleu_1: 0.640
Bleu_2: 0.453
Bleu_3: 0.310
Bleu_4: 0.204
computing METEOR score...
METEOR: 0.192
computing Rouge score...
ROUGE_L: 0.466
computing CIDEr score...
CIDEr: 0.573


In [17]:
# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print('%s: %.3f'%(metric, score))

Bleu_1: 0.640
Bleu_2: 0.453
Bleu_3: 0.310
Bleu_4: 0.204
METEOR: 0.192
ROUGE_L: 0.466
CIDEr: 0.573
